# ICUsICS DB tutorial

ICUsICS is an anonymized database built from the data stored into the Clinical Information System (CIS) database of 6 Intensive Care Units (ICUs) from the Catalan Institute of Health (ICS). Actually, it is a database of databases, because each ICU belongs to a different hospital and each CIS presents its own particularities. However, the tables architecture of each database is identical between hospitals, which facilitates data search and extraction.

ICUsICS is not hosted as a database 'per se', but as a directory of folders (tables) with csv files inside (registries). Inside icuics-db, there are 6 folders, 1 for each hospital, and inside them there are 10 folders:  

patients: it contains patient-level info (id, hospital, demografics and admission and discharge time and wards)  
variables_ref: it contains info of the variables present in the database (id, hospital, name, type). Key info: vartype 1, 2, 4 and 8 mean v_monitored, v_labres, v_observed and v_derived respectivelly (the tables where the variable is stored). Key info: datatype 0, 1 and 2 mean numeric, categoric and checkbox respectivelly.  
v_monitored: table with registries for vartype 1 variables  
v_labres: table with registries for vartype 2 variables  
v_observed: table with registries for vartype 4 variables  
v_derived: table with registries for vartype 8 variables  
drugs_ref: it contains info of the drugs present in the database (id, hospital, name, formunit, unit, etc.)  
drugs: table with registries for drugs  
diagnoses: table with the diagnoses  
insertions: table with the insertions  

## Example of cohort preparation

Let's create a cohort that accomplishes the following inclusion/exclusion criteria:  
Patients 

In [11]:
dp = {'database': ['icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo','icusics-db-demo'],
      'hospital': ['h3','h3','h3','h3','h3','h3','h3','h3','h3','h3'],
      'table': ['patients','variables_ref','v_monitored','v_labres','v_observed','v_derived','drugs_ref','drugs','diagnoes','insertions']}
database_paths = pd.DataFrame(data=dp)

,database,hospital,table
0,icusics-db-demo,h3,patients
1,icusics-db-demo,h3,variables_ref
2,icusics-db-demo,h3,v_monitored
3,icusics-db-demo,h3,v_labres
4,icusics-db-demo,h3,v_observed
5,icusics-db-demo,h3,v_derived
6,icusics-db-demo,h3,drugs_ref
7,icusics-db-demo,h3,drugs
8,icusics-db-demo,h3,diagnoes
9,icusics-db-demo,h3,insertions


In [17]:
database_paths[database_paths['table']=='v_observed']['database'].values

array(['icusics-db-demo'], dtype=object)

In [ ]:
with open('filename', 'wb') as data:
    bucket.download_fileobj('mykey', data)

In [98]:
def search_for(db, h, t, k):
    
    df = pd.read_csv(f's3://{db}/{h}_db/{t}/{h}_{t}.csv')
    
    if 'drugs' in t:
        
        i = ''
        
        try:
            return df[df['pharmaname'].str.contains(k, case=False, na=False)]
        except:
            return print('key_field not found')
        
    ifelse 'variables' in t


In [102]:
result = search_for('icusics-db-demo', 'h3', 'drugs_ref', 'pepito')

In [103]:
result['a_pharmaid']

Series([], Name: a_pharmaid, dtype: int64)

In [81]:
caca.head()

,a_pharmaid,hospital_coded,pharmaname,pharmagroupname,pharmaformunit,pharmadoseunit,pharmadoseformratio,pharmavolumeunit,pharmavolumeformratio
0,3000000004,3,COMPLEX DE PROTROMBINA 600 UI (F. IX) vi,.Antihemorràgics,vial,UI,600.0,ml,0.0
1,3000000005,3,OMEPRAZOL 40 mg vial,.Antiàcids,vial,mg,40.0,ml,0.0
2,3000000021,3,HEPARINA SODICA 5.000 UI/5 ml (1%) vial,ANTITROMBOTICS,ml,UI,1000.0,ml,1.0
3,3000000025,3,FOSFAT MONOPOTASSIC 10 mmol P/10 ml (1 M,.Electròlits,ml,mmol,1.0,ml,1.0
4,3000000049,3,FUROSEMIDA 250 mg/25 ml amp,DIURETICS,ml,mg,10.0,ml,1.0


In [65]:
def fetch_data(db, h, t):
    
    df = pd.DataFrame()
    
    bucket = s3.Bucket(db)

    for my_bucket_object in bucket.objects.all():
    
        if all(x in my_bucket_object.key for x in [h, t]):
            
            chunk = pd.read_csv(f's3://{db}/{my_bucket_object.key}')
            df_chunk = chunk[(chunk['a_variableid']==3015002262) & (chunk['value']==12)]
            df = pd.concat([df, df_chunk])
            
    return df

#     tp = f's3://{db}/{h}_db/{t}'
    
#     folder = os.listdir(tp)
#     files = [match for match in folder if f"{h}_" in match]

#     vmi = pd.DataFrame()

#     for file in files:
    
#         chunk = pd.read_csv(f'{f}/{file}/')
#         vmi_chunck = chunk[(chunk['a_variableid']==3015002262) & (chunk['value']==12)]
#         vmi = pd.concat([vmi, vmi_chunck])
        
#     return vmi

In [66]:
caca = fetch_data('icusics-db-demo', 'h3', 'v_observed')

In [69]:
caca.shape
print(caca['a_patientid'].nunique())

3223


In [22]:
caca = fetch_data('icusics-db-demo','h3','v_observed')

FileNotFoundError: [Errno 2] No such file or directory: 's3://icusics-db-demo/h3_db/v_observed'

In [44]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
role = get_execution_role()
s3 = boto3.resource('s3')

In [3]:
bucket='icusics-db-demo'
file_key = 'h3_db/drugs_ref/h3_drugs_ref.csv'
s3uri = 's3://{}/{}'.format(bucket, file_key)
df = pd.read_csv(s3uri)
df.head()

,a_pharmaid,hospital_coded,pharmaname,pharmagroupname,pharmaformunit,pharmadoseunit,pharmadoseformratio,pharmavolumeunit,pharmavolumeformratio
0,3000000004,3,COMPLEX DE PROTROMBINA 600 UI (F. IX) vi,.Antihemorràgics,vial,UI,600.0,ml,0.0
1,3000000005,3,OMEPRAZOL 40 mg vial,.Antiàcids,vial,mg,40.0,ml,0.0
2,3000000021,3,HEPARINA SODICA 5.000 UI/5 ml (1%) vial,ANTITROMBOTICS,ml,UI,1000.0,ml,1.0
3,3000000025,3,FOSFAT MONOPOTASSIC 10 mmol P/10 ml (1 M,.Electròlits,ml,mmol,1.0,ml,1.0
4,3000000049,3,FUROSEMIDA 250 mg/25 ml amp,DIURETICS,ml,mg,10.0,ml,1.0


In [6]:
df['vartype'].value_counts()

4    5023
2     424
1     328
8     218
Name: vartype, dtype: int64

In [7]:
df['datatype'].value_counts()

1    4597
0     974
2     422
Name: datatype, dtype: int64

In [ ]:
%%time

h = '3'
f = f'h{h}_db/v_derived'

folder = os.listdir(f)
files = [match for match in folder if f"h{h}_" in match]

vmi = pd.DataFrame()
chunk_index=1

for file in files:
    
    chunk = pd.read_csv(f'{f}/{file}')
    vmi_chunck = chunk[(chunk['a_variableid']==3015002262) & (chunk['value']==12)]
    vmi = pd.concat([vmi, vmi_chunck])